# Spatial & Geospatial Data Preprocessing in Python
#### Introduction
#### Spatial and geospatial data contain location-based information that represents objects, events, or phenomena tied to the Earth's surface.

#### Spatial data includes objects with coordinates (e.g., points, lines, polygons).
#### Geospatial data contains location + attributes (e.g., population density, weather data).
#### Examples of Spatial & Geospatial Data Applications:
#### ✅ Maps & Navigation – GPS, Google Maps
#### ✅ Climate & Environment – Weather forecasting
#### ✅ Urban Planning – Infrastructure and transportation
#### ✅ Remote Sensing – Satellite imagery analysis
#### 
#### 1️⃣ Spatial Data Representation & Formats
#### Geospatial data can be stored in different formats:
#### 
#### Vector Data (Points, Lines, Polygons) → Used for locations, roads, buildings
#### Raster Data (Gridded pixels) → Used for satellite images, elevation models
#### Geospatial Databases: PostGIS, SpatiaLite, MongoDB (GeoJSON)
#### 📌 Example: Loading Geospatial Data in Python


In [ ]:
import geopandas as gpd

# Load a shapefile (vector data)
gdf = gpd.read_file("path/to/shapefile.shp")

# Display first rows
print(gdf.head())

# Plot the spatial data
gdf.plot()

#### 2️⃣ Coordinate Reference Systems (CRS) & Reprojection
#### Why?
#### Geospatial data is often collected in different CRS (Coordinate Reference Systems).
#### Reprojection is needed to align different datasets for spatial analysis.
#### 📌 Example: Reprojecting to a Common CRS

In [ ]:
# Check current CRS
print(gdf.crs)

# Convert to a different CRS (e.g., WGS 84)
gdf = gdf.to_crs(epsg=4326)


#### Comparison of CRS:
#### 
#### CRS	Description	Best For
#### EPSG:4326	WGS 84 (lat, lon)	GPS, Google Maps
#### EPSG:3857	Web Mercator	Web mapping (OpenStreetMap)
#### EPSG:32633	UTM Zone 33N	Local, high-accuracy mapping
#### 3️⃣ Handling Missing or Inaccurate Location Data
#### Techniques:
#### Remove invalid coordinates (out of bounds).
#### Interpolate missing values (e.g., using neighboring points).
#### Replace missing coordinates with nearest valid location.
#### 📌 Example: Removing Invalid Coordinates

In [ ]:

# Remove rows where latitude or longitude is missing
gdf = gdf.dropna(subset=["latitude", "longitude"])

# Filter data within valid longitude/latitude range
gdf = gdf[(gdf.longitude >= -180) & (gdf.longitude <= 180) &
          (gdf.latitude >= -90) & (gdf.latitude <= 90)]


#### 4️⃣ Spatial Data Cleaning & Normalization
#### Common Cleaning Steps:
#### Remove duplicate geometries
#### Fix invalid geometries (self-intersections, gaps)
#### Convert inconsistent data types
#### 📌 Example: Fixing Invalid Geometries

In [ ]:
from shapely.validation import make_valid

# Fix invalid geometries
gdf["geometry"] = gdf["geometry"].apply(make_valid)

#### 5️⃣ Geospatial Feature Engineering
#### Spatial features improve machine learning models.
#### 
#### (i) Distance Calculation
#### Measure distance between points for applications like nearest neighbor search, travel time estimation.
#### 
#### 📌 Example: Compute Distance Between Two Locations

In [ ]:
from geopy.distance import geodesic

# Define two locations (lat, lon)
loc1 = (40.7128, -74.0060)  # New York
loc2 = (34.0522, -118.2437)  # Los Angeles

# Calculate distance in km
distance_km = geodesic(loc1, loc2).km
print(f"Distance: {distance_km:.2f} km")

#### (ii) Spatial Joins
#### Combine datasets based on location.
#### 
#### 📌 Example: Join Points to Polygons

In [ ]:
# Load city boundaries
city_boundaries = gpd.read_file("city_boundaries.shp")

# Perform a spatial join
points_with_city = gpd.sjoin(gdf, city_boundaries, how="left", predicate="within")

#### (iii) Extracting Features from Spatial Data
#### Feature	Description	Example Usage
#### Proximity	Distance to nearest feature	Nearest hospital, road
#### Spatial Density	Count of features in a given area	Population density
#### Elevation	Height above sea level	Flood prediction
#### 📌 Example: Compute Nearest Road Distance

In [ ]:
from scipy.spatial import cKDTree

# Convert to NumPy arrays
point_coords = gdf[["longitude", "latitude"]].values
road_coords = roads[["longitude", "latitude"]].values

# Build KDTree for fast nearest-neighbor search
tree = cKDTree(road_coords)

# Compute distance to nearest road
distances, _ = tree.query(point_coords)
gdf["nearest_road_distance"] = distances

#### 6️⃣ Raster Data Processing
#### Raster data (satellite images, elevation data) needs preprocessing for machine learning.
#### 
#### 📌 Example: Load and Process Satellite Image (Raster)

In [ ]:
import rasterio

# Open raster file
with rasterio.open("satellite.tif") as dataset:
    image = dataset.read(1)  # Read first band
    print(image.shape)


#### 7️⃣ Spatial Data Transformation
#### Convert Vector Data to Grid (Rasterization)

In [ ]:
from rasterio.features import rasterize

# Convert vector polygons to raster
rasterized = rasterize(shapes=gdf.geometry, out_shape=(100, 100))

#### Convert Raster Data to Points

In [ ]:
import numpy as np

# Extract values from raster
rows, cols = np.where(image > 0)
values = image[rows, cols]

#### Comparison of Spatial Preprocessing Techniques
#### Method	Purpose	Best For	Pros	Cons
#### CRS Reprojection	Convert spatial data to a standard CRS	Merging datasets	Necessary for accuracy	Can introduce distortions
#### Missing Data Handling	Fix missing locations	GPS data cleaning	Improves completeness	May introduce bias
#### Spatial Joins	Merge datasets spatially	Adding context	Enhances dataset	Computational cost
#### Feature Engineering (Distance, Density)	Add meaningful features	ML models	Boosts performance	Feature selection needed
#### Raster Processing	Handle images/satellite data	Remote sensing, climate data	Richer representation	Large file sizes
#### Final Thoughts
#### Spatial data preprocessing is crucial for GIS, remote sensing, and ML applications.
#### Coordinate transformation and data cleaning ensure consistency.
#### Feature engineering (distances, densities, spatial joins) boosts machine learning models.
#### Raster and vector processing enable advanced analysis.
#### Would you like a deep dive into spatial machine learning techniques (e.g., spatial clustering, geospatial deep learning)? 🚀

#### Advanced Spatial & Geospatial Data Preprocessing
#### In this section, we will cover advanced geospatial data preprocessing techniques with detailed explanations, Python examples, and comparisons.
#### 
#### 1️⃣ Spatial Aggregation
#### 🔹 What is it?
#### Spatial aggregation groups raw point-based data (latitude, longitude) into larger spatial units like:
#### 
#### Zip codes, city districts, or states
#### Hexagonal or square grids
#### Administrative boundaries
#### 🔹 Why use Spatial Aggregation?
#### ✅ Reduces the granularity of data, making it easier to analyze
#### ✅ Improves interpretability by summarizing dense spatial data
#### ✅ Helps in data privacy by avoiding revealing exact locations
#### 
#### 📌 Example: Aggregating Data by Zip Code

In [ ]:

import geopandas as gpd
import pandas as pd

# Load raw point data (e.g., customer locations)
points = gpd.read_file("customer_locations.shp")

# Load zip code boundaries (polygon shapefile)
zip_codes = gpd.read_file("zip_codes.shp")

# Spatial join: Assign each point to a zip code
points_with_zip = gpd.sjoin(points, zip_codes, how="left", predicate="within")

# Aggregate customer count per zip code
zip_agg = points_with_zip.groupby("zip_code")["customer_id"].count().reset_index()
zip_agg.rename(columns={"customer_id": "customer_count"}, inplace=True)

print(zip_agg.head())


#### 🆚 Comparison of Aggregation Methods
#### Method	Best For	Pros	Cons
#### Zip Codes / Administrative Boundaries	Census, business analysis	Meaningful real-world areas	Irregular shape sizes
#### Hexagonal Grids (H3, Uber)	Climate, mobility data	Equal area, avoids bias	Computational overhead
#### Square Grids (Fishnet)	Raster-based aggregation	Simple and fast	May not align with natural boundaries
#### 2️⃣ Feature Engineering for Geospatial Data
#### 🔹 What is it?
#### Feature engineering extracts meaningful numerical features from spatial data to enhance machine learning models.
#### 
#### 🔹 Common Spatial Features
#### ✅ Distance to nearest key locations (e.g., distance to nearest hospital, store, road)
#### ✅ Spatial Autocorrelation (e.g., clustering of similar data values)
#### ✅ Density metrics (e.g., number of points in a given area)
#### 
#### 📌 Example: Compute Distance to Nearest Hospital

In [ ]:
from scipy.spatial import cKDTree
import numpy as np

# Convert hospital and customer locations to NumPy arrays
hospitals_coords = hospitals[["longitude", "latitude"]].values
customer_coords = customers[["longitude", "latitude"]].values

# Build KDTree for fast nearest neighbor search
tree = cKDTree(hospitals_coords)

# Compute distance to nearest hospital
distances, _ = tree.query(customer_coords)
customers["nearest_hospital_distance"] = distances

print(customers.head())


#### 📌 Example: Compute Spatial Autocorrelation (Moran’s I)
#### Spatial autocorrelation measures how similar values are across space.
#### 
#### 


In [ ]:
from esda.moran import Moran
import libpysal as ps

# Create spatial weights matrix (based on neighbors)
w = ps.weights.KNN.from_dataframe(gdf, k=5)

# Compute Moran's I (spatial autocorrelation)
moran = Moran(gdf["price"], w)
print(f"Moran's I: {moran.I}")


#### 🆚 Comparison of Spatial Feature Engineering Methods
#### Method	Usage	Pros	Cons
#### Distance to Nearest Point (KDTree)	Urban planning, travel time estimation	Fast & scalable	Needs preprocessing
#### Density Metrics (Kernel Density Estimation)	Crime mapping, customer clustering	Captures spatial intensity	Sensitive to bandwidth selection
#### Spatial Autocorrelation (Moran’s I, Geary’s C)	Pattern detection, clustering	Measures spatial similarity	Needs statistical interpretation
#### 3️⃣ Geospatial Interpolation
#### 🔹 What is it?
#### Geospatial interpolation estimates missing or unknown values at unobserved locations using data from known points.
#### 
#### 🔹 Why use it?
#### ✅ Fills missing data gaps (e.g., estimating missing temperature values)
#### ✅ Creates continuous surfaces from scattered point data
#### ✅ Essential for environmental modeling (e.g., pollution, rainfall mapping)
#### 
#### Interpolation Methods
#### Method	Usage	Pros	Cons
#### Inverse Distance Weighting (IDW)	Quick spatial estimation	Simple & fast	Assumes local influence
#### Kriging (Ordinary, Universal, etc.)	Advanced geostatistical modeling	Accounts for spatial trends	Computationally intensive
#### Spline Interpolation	Smooth surfaces	Good for gradual changes	Can oversmooth sharp changes
#### 📌 Example: Inverse Distance Weighting (IDW)

In [ ]:
import numpy as np
from scipy.spatial import distance

# Define known data points (coordinates & values)
known_points = np.array([[0, 0, 10], [1, 1, 15], [2, 2, 20]])
unknown_point = np.array([1.5, 1.5])  # Location where we estimate value

# Compute inverse distance weights
dists = distance.cdist([unknown_point], known_points[:, :2], metric='euclidean')
weights = 1 / dists
weights /= weights.sum()

# Compute estimated value
estimated_value = np.sum(weights * known_points[:, 2])
print(f"Estimated value at {unknown_point}: {estimated_value:.2f}")


#### 📌 Example: Kriging Interpolation using PyKrige

In [ ]:
from pykrige.ok import OrdinaryKriging
import numpy as np

# Known points (x, y, value)
x = np.array([0, 1, 2])
y = np.array([0, 1, 2])
values = np.array([10, 15, 20])

# Create Kriging model
OK = OrdinaryKriging(x, y, values, variogram_model="spherical")

# Predict at new point
z, ss = OK.execute("points", np.array([1.5]), np.array([1.5]))
print(f"Kriging estimated value: {z[0]:.2f}")


#### 🆚 IDW vs. Kriging: When to Use Each?
#### Method	Best For	Pros	Cons
#### IDW	Quick estimates with local influence	Simple & fast	Doesn't model global trends
#### Kriging	Complex spatial relationships	Statistically rigorous	Computationally expensive
#### Final Thoughts
#### ✅ Spatial aggregation is useful for simplifying raw coordinate data into meaningful regions
#### ✅ Feature engineering enhances predictive models by adding spatial relationships
#### ✅ Geospatial interpolation fills missing values and creates continuous spatial surfaces